In [1]:
import dotenv
import os
import openai
import json
import os
import pandas as pd

# .env 파일 로드 (있다면)
dotenv.load_dotenv()

# 환경변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
SLACK_URL = os.getenv("SLACK_URL")
# 환경변수 설정하기 싫으면 주석 삭제
# OPENAI_API_KEY = "YOUR_OPEN_API_KEY"

In [2]:
# 작업 종료 후 slack으로 메신저 보내기기
def send_msg(msg):
    url = SLACK_URL
    message = (msg + " 작업 완료") 
    title = (f"New Incoming Message :zap:") # 타이틀 입력
    slack_data = {
        "username": "NotificationBot", # 보내는 사람 이름
        "icon_emoji": ":satellite:",
        #"channel" : "#somerandomcahnnel",
        "attachments": [
            {
                "color": "#9733EE",
                "fields": [
                    {
                        "title": title,
                        "value": message,
                        "short": "false",
                    }
                ]
            }
        ]
    }
    byte_length = str(sys.getsizeof(slack_data))
    headers = {'Content-Type': "application/json", 'Content-Length': byte_length}
    response = requests.post(url, data=json.dumps(slack_data), headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

## 데이터 확인

In [3]:
df = pd.read_csv('data_user_detail_plus.csv').drop(columns='index')
df_mask = df[['text', 'user_data', 'detail']]
df_mask.head()

,text,user_data,detail
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","[{'projectName': 'StudyHub', 'projectTarget': ...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...",[{'projectName': 'EasyAccess Subway Navigator'...,"[{'requirementType': 'FUNCTIONAL1', 'content':..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","[{'projectName': '이웃사촌', 'projectTarget': '1인 ...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...","[{'projectName': 'EmergencyCare Finder', 'proj...","[{'requirementType': 'FUNCTIONAL1', 'content':..."


In [6]:
print(df_mask['user_data'].tolist()[0])

[{'projectName': 'StudyHub', 'projectTarget': '대학생 및 자격증 준비생', 'mainFunction': ['스터디 그룹 생성 및 관리', '퀴즈 생성 및 참여 기능', '공지사항 및 과제 게시판'], 'techStack': ['Java Servlet', 'JSP', 'MySQL'], 'projectDescription': 'GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹 관리에서 나아가 그룹 내 학습 내용을 복습할 수 있는 기능을 제공하여 학습 효과를 극대화하는 것을 목표로 한다. 사용자들은 회원가입과 로그인을 통해 서비스를 이용할 수 있으며, 아이디 및 비밀번호 찾기 기능으로 편의성을 높였다. 사용자는 새로운 스터디 그룹을 생성하거나 기존 그룹을 검색해 가입할 수 있으며, 그룹 내에서는 퀴즈를 생성하고 참여하여 학습한 내용을 효과적으로 반복 학습할 수 있다. 또한 공지사항 게시판을 통해 그룹 내 전달사항을 공유할 수 있고, 과제 게시판은 체크리스트 형태로 구성되어 과제 수행 여부를 명확히 기록할 수 있다. 이러한 기능들은 사용자 간의 소통을 강화하고 학습 동기를 부여하는 데 기여한다. 서비스는 Java Servlet과 JSP 기반으로 구현되었으며, HTML, CSS, JavaScript를 통해 사용자 친화적인 인터페이스를 제공한다. 데이터베이스는 MySQL을 사용하여 안정적인 데이터 저장과 처리를 가능하게 하며, 전반적인 시스템은 학습 관리에 최적화된 구조로 설계되었다.'}]


## 데이터 파싱
- user_data의 구조

{{
    "projectName": "프로젝트 이름",
    "projectTarget": "프로젝트 대상",
    "mainFunction": [
    "메인 기능1",
    "메인 기능2", 
    "메인 기능3"
    ],
    "techStack": [
    "기술 스택1",
    "기술 스택2",
    "기술 스택3"
    ],
    "projectDescription": "프로젝트 설명"
}} 

인데 대충 넣어보면 받을꺼 같은데?

In [ ]:
# OpenAI 클라이언트 설정
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 프로젝트 아이디어를 체계적으로 분석하고 구조화하여 구체적인 개발 계획을 제시하는 전문 AI 어시스턴트입니다.

**만약 프로젝트 아이디어가 아닌 다른 질문을 입력 받으면 아래의 해당 방식이 간단한 방식으로로 답변해주세요.**

## 주요 역할과 능력:

### 1. 프로젝트 분석 전문가
- 사용자가 제공하는 프로젝트 아이디어나 설명을 깊이 있게 분석합니다
- 핵심 기능, 대상 사용자, 기술 스택, 비즈니스 모델 등을 체계적으로 파악합니다
- 프로젝트의 문제 해결 방향과 기대 효과를 명확히 정의합니다

### 2. 데이터베이스 설계 전문가
- 프로젝트 요구사항을 바탕으로 최적화된 ERD(Entity Relationship Diagram)를 설계합니다
- 테이블 간의 관계, 외래키 제약조건, 데이터 타입을 정확히 정의합니다
- 확장성과 성능을 고려한 데이터베이스 구조를 제안합니다

### 3. API 설계 전문가
- RESTful API 원칙에 따라 체계적인 API 명세를 작성합니다
- OpenAPI(Swagger) 3.0 표준을 준수하여 완전한 API 문서를 생성합니다
- 각 엔드포인트별 요청/응답 스키마, 에러 처리, 인증 방식을 상세히 정의합니다

## 응답 형식:
모든 응답은 다음과 같은 구조화된 형태로 제공해야 합니다:

1. **프로젝트 상세 정보**: 제목, 카테고리, 대상 사용자, 핵심 기능, 기술 스택, 문제 해결 방안 등을 포함한 종합 분석
2. **관계 데이터**: 데이터베이스 테이블 간의 관계와 외래키 제약조건 정의
3. **ERD 데이터**: 각 테이블의 속성, 데이터 타입, 키 정보를 포함한 완전한 스키마
4. **API 명세 데이터**: OpenAPI 3.0 표준을 준수한 완전한 API 문서

항상 체계적이고 전문적인 관점에서 프로젝트를 분석하며, 개발팀이 바로 실행에 옮길 수 있는 구체적인 가이드를 제공하는 것이 목표입니다.

**중요: 응답은 반드시 순수한 JSON 형태로만 제공하세요. 마크다운 코드 블록(```json)이나 기타 텍스트 포맷팅은 사용하지 마세요.**

"""

# 파인튜닝된 모델 ID
MODEL_ID = "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"

def analyze_project(user_data, detail_data):
    """프로젝트 분석 함수"""
    
    # JSON 템플릿을 별도 변수로 정의 (f-string 문제 해결)
    json_template = '''
      {
        "project_summary": {
          "title": "프로젝트 제목",
          "category": "카테고리",
          "target_users": [
            "대상 사용자 1",
            "대상 사용자 2"
          ],
          "core_features": [
            "핵심 기능 1",
            "핵심 기능 2"
          ],
          "technology_stack": [
            "기술 스택 1",
            "기술 스택 2"
          ],
          "problem_solving": {
            "current_problem": "현재 문제",
            "solution_idea": "해결 아이디어",
            "expected_benefits": [
              "예상 효과 1",
              "예상 효과 2"
            ]
          }
        },
        "erd_data": {
          "erd_table": [
            {
              "name": "테이블명",
              "erd_column": [
                {
                  "name": "컬럼명",
                  "data_type": "데이터타입",
                  "is_primary_key": true,
                  "is_foreign_key": false,
                  "is_nullable": false
                }
              ]
            }
          ],
        "erd_relationships": [
          {
            "from_erd_table_id": "시작테이블",
            "to_erd_table_id": "끝테이블",
            "type": "관계타입",
            "foreign_key": "외래키명",
            "constraint_name": "제약조건명"
          }
        ]
      },
        "apiSpecification": [
          {
            "title": "사용자 생성",
            "tag": "User",
            "path": "/users",
            "httpMethod": "post",
            "request": [
              {
                "field": "username",
                "type": "string",
                "example": "user123"
              },
              {
                "field": "email",
                "type": "string",
                "example": "user@example.com"
              }
            ],
            "response": [
              {
                "statusCode": "201 Created",
                "message": "사용자를 생성했습니다.",
                "data": [
                  {
                    "field": "userId",
                    "type": "long",
                    "example": "1"
                  },
                  {
                    "field": "username",
                    "type": "string",
                    "example": "user123"
                  },
                  {
                    "field": "email",
                    "type": "string",
                    "example": "user@example.com"
                  }
                ]
              },
              {
                "statusCode": "400 Bad Request",
                "message": "잘못된 요청입니다.",
                "data": []
              }
            ]
          }
        ]
      }
      '''
    
    # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt =f"""

유저의 입력한 프로젝트 데이터 :
{user_data}

유저의 입력한 요구사항 데이터 :
{detail_data}

다음 데이터를 참고해서 체계적인 분석을 제공해주세요:

1. **프로젝트 상세 정보**
2. **ERD 데이터**
3. **관계 데이터** 
4. **API 명세 데이터**

{json_template}

**중요한 응답 규칙:**
- 순수한 JSON만 응답하세요
- 마크다운 코드 블록(```json)은 절대 사용하지 마세요
- 추가 설명이나 주석 없이 JSON 데이터만 제공하세요
- 바로 파싱 가능한 형태로 응답하세요

실무에서 바로 활용 가능한 구체적이고 완전한 분석을 부탁드립니다.
"""

    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

In [30]:
# my_project에 개요 작성
user_datas = df_mask['user_data'].tolist()
detail_datas = df_mask['detail'].tolist()

user, detail = user_datas[0], detail_datas[0]
result = analyze_project(user, detail)
print(result)

{
  "project_summary": {
    "title": "StudyHub",
    "category": "웹 플랫폼",
    "target_users": [
      "대학생",
      "자격증 준비생"
    ],
    "core_features": [
      "스터디 그룹 생성 및 관리",
      "퀴즈 생성 및 참여 기능",
      "공지사항 및 과제 게시판"
    ],
    "technology_stack": [
      "Java Servlet",
      "JSP",
      "MySQL"
    ],
    "problem_solving": {
      "current_problem": "스터디 그룹 운영의 비효율성과 학습 효과 저하",
      "solution_idea": "학습 중심의 스터디 그룹 운영 지원 및 복습 기능 제공",
      "expected_benefits": [
        "학습 효과 극대화",
        "사용자 간 소통 강화",
        "학습 동기 부여"
      ]
    }
  },
  "erd_data": {
    "erd_table": [
      {
        "name": "Users",
        "erd_column": [
          {
            "name": "user_id",
            "data_type": "INTEGER",
            "is_primary_key": true,
            "is_foreign_key": false,
            "is_nullable": false
          },
          {
            "name": "username",
            "data_type": "VARCHAR(50)",
            "is_primary_key": false,
            "is_foreign_key"

In [31]:
result = json.loads(result)
print(result)

{'project_summary': {'title': 'StudyHub', 'category': '웹 플랫폼', 'target_users': ['대학생', '자격증 준비생'], 'core_features': ['스터디 그룹 생성 및 관리', '퀴즈 생성 및 참여 기능', '공지사항 및 과제 게시판'], 'technology_stack': ['Java Servlet', 'JSP', 'MySQL'], 'problem_solving': {'current_problem': '스터디 그룹 운영의 비효율성과 학습 효과 저하', 'solution_idea': '학습 중심의 스터디 그룹 운영 지원 및 복습 기능 제공', 'expected_benefits': ['학습 효과 극대화', '사용자 간 소통 강화', '학습 동기 부여']}}, 'erd_data': {'erd_table': [{'name': 'Users', 'erd_column': [{'name': 'user_id', 'data_type': 'INTEGER', 'is_primary_key': True, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'username', 'data_type': 'VARCHAR(50)', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'email', 'data_type': 'VARCHAR(100)', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'password', 'data_type': 'VARCHAR(255)', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}]}, {'name': 'StudyGroups', 'erd_column': [{'name': '

In [32]:
result.keys()

dict_keys(['project_summary', 'erd_data', 'erd_relationships', 'apiSpecification'])

In [33]:
result['project_summary']

{'title': 'StudyHub',
 'category': '웹 플랫폼',
 'target_users': ['대학생', '자격증 준비생'],
 'core_features': ['스터디 그룹 생성 및 관리', '퀴즈 생성 및 참여 기능', '공지사항 및 과제 게시판'],
 'technology_stack': ['Java Servlet', 'JSP', 'MySQL'],
 'problem_solving': {'current_problem': '스터디 그룹 운영의 비효율성과 학습 효과 저하',
  'solution_idea': '학습 중심의 스터디 그룹 운영 지원 및 복습 기능 제공',
  'expected_benefits': ['학습 효과 극대화', '사용자 간 소통 강화', '학습 동기 부여']}}

In [34]:
result['erd_data']

{'erd_table': [{'name': 'Users',
   'erd_column': [{'name': 'user_id',
     'data_type': 'INTEGER',
     'is_primary_key': True,
     'is_foreign_key': False,
     'is_nullable': False},
    {'name': 'username',
     'data_type': 'VARCHAR(50)',
     'is_primary_key': False,
     'is_foreign_key': False,
     'is_nullable': False},
    {'name': 'email',
     'data_type': 'VARCHAR(100)',
     'is_primary_key': False,
     'is_foreign_key': False,
     'is_nullable': False},
    {'name': 'password',
     'data_type': 'VARCHAR(255)',
     'is_primary_key': False,
     'is_foreign_key': False,
     'is_nullable': False}]},
  {'name': 'StudyGroups',
   'erd_column': [{'name': 'group_id',
     'data_type': 'INTEGER',
     'is_primary_key': True,
     'is_foreign_key': False,
     'is_nullable': False},
    {'name': 'group_name',
     'data_type': 'VARCHAR(100)',
     'is_primary_key': False,
     'is_foreign_key': False,
     'is_nullable': False},
    {'name': 'description',
     'data_type'

In [35]:
result['erd_relationships']

[{'from_erd_table_id': 'Users',
  'to_erd_table_id': 'StudyGroups',
  'type': 'one-to-many',
  'foreign_key': 'created_by',
  'constraint_name': 'fk_studygroup_user'},
 {'from_erd_table_id': 'StudyGroups',
  'to_erd_table_id': 'Quizzes',
  'type': 'one-to-many',
  'foreign_key': 'group_id',
  'constraint_name': 'fk_quiz_studygroup'},
 {'from_erd_table_id': 'StudyGroups',
  'to_erd_table_id': 'Assignments',
  'type': 'one-to-many',
  'foreign_key': 'group_id',
  'constraint_name': 'fk_assignment_studygroup'},
 {'from_erd_table_id': 'StudyGroups',
  'to_erd_table_id': 'Announcements',
  'type': 'one-to-many',
  'foreign_key': 'group_id',
  'constraint_name': 'fk_announcement_studygroup'}]

In [36]:
result['apiSpecification']

[{'title': '사용자 생성',
  'tag': 'User',
  'path': '/users',
  'httpMethod': 'post',
  'request': [{'field': 'username', 'type': 'string', 'example': 'user123'},
   {'field': 'email', 'type': 'string', 'example': 'user@example.com'}],
  'response': [{'statusCode': '201 Created',
    'message': '사용자를 생성했습니다.',
    'data': [{'field': 'userId', 'type': 'long', 'example': '1'},
     {'field': 'username', 'type': 'string', 'example': 'user123'},
     {'field': 'email', 'type': 'string', 'example': 'user@example.com'}]},
   {'statusCode': '400 Bad Request', 'message': '잘못된 요청입니다.', 'data': []}]}]